In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import xgboost as xgb
from sklearn.impute import SimpleImputer
from glob import glob
import os
from tqdm import tqdm
from hyperopt import fmin, tpe, hp, Trials
from hyperopt.pyll.base import scope
from scipy.optimize import OptimizeWarning
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder

import statsmodels.api as sm

import numpy as np
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.discrete.discrete_model import NegativeBinomialP, Poisson
from statsmodels.discrete.discrete_model import NegativeBinomial
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=OptimizeWarning)

ARIMA, Exponential Smoothing, Simple Exponential Smoothing, Negative Binomial Autoregressive, and Poisson Autoregressive

### Load Files

In [ ]:
subject_files = glob("/content/drive/MyDrive/Upwork clients/ACCOUNT-CREATOR/DiaTrend Data/DiaTrend Data/Sensor Data/*.xlsx")
print(f"Number of subject files found: {len(subject_files)}")

Number of subject files found: 54


##### Configuration
##### **Note**: Set `Subjects` and `CGM_Inputs` in Conf class below as per your computation power

In [ ]:
class Conf :
    Subjects = 54 # define how many subjects you want to include in processing
    CGM_Inputs = 100000 # define how many CGM inputs (rows) to consider for training, for all samples use None

### Data Preprocessing

In [ ]:
# Placeholder for all data
all_data = []

# Iterate over each subject file
for subject_file in tqdm(sorted(subject_files)[:min(Conf.Subjects, len(subject_files))]):
    try:
        print(f"Processing {subject_file}...")

        # Load CGM and Bolus data for each subject
        cgm_data = pd.read_excel(subject_file, sheet_name='CGM', parse_dates=['date'])

        # Check if data was loaded correctly
        if cgm_data.empty:
            print(f"No data found in {subject_file}")
            continue

        cgm_data['date'] = pd.to_datetime(cgm_data['date'], format='ISO8601')

        # Append to the all_data list
        if Conf.CGM_Inputs is None:
            all_data.append(cgm_data.head(len(cgm_data)))
        else:
            all_data.append(cgm_data.head(min(Conf.CGM_Inputs, len(cgm_data))))

    except Exception as ex:
        print(f"Error Occurred at {os.path.basename(subject_file)}: {str(ex)}")


# Check if all_data is empty before concatenation
if not all_data:
    print("No data was successfully processed. Exiting.")
else:
    # Concatenate all subjects' data into a single DataFrame
    final_data = pd.concat(all_data, axis=0).reset_index(drop=True)
    X = final_data['date']
    y = final_data['mg/dl']
    print("Dataset shape:", X.shape)

  0%|          | 0/54 [00:00<?, ?it/s]

Processing /content/drive/MyDrive/Upwork clients/ACCOUNT-CREATOR/DiaTrend Data/DiaTrend Data/Sensor Data/Subject1.xlsx...


  2%|▏         | 1/54 [00:36<31:50, 36.04s/it]

Processing /content/drive/MyDrive/Upwork clients/ACCOUNT-CREATOR/DiaTrend Data/DiaTrend Data/Sensor Data/Subject10.xlsx...


  4%|▎         | 2/54 [00:49<19:29, 22.50s/it]

...

Processing /content/drive/MyDrive/Upwork clients/ACCOUNT-CREATOR/DiaTrend Data/DiaTrend Data/Sensor Data/Subject8.xlsx...


 98%|█████████▊| 53/54 [05:44<00:11, 11.54s/it]

Processing /content/drive/MyDrive/Upwork clients/ACCOUNT-CREATOR/DiaTrend Data/DiaTrend Data/Sensor Data/Subject9.xlsx...


100%|██████████| 54/54 [05:59<00:00,  6.66s/it]

Dataset shape: (2027648,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
print(X_train.shape, y_train.shape)

(1622118,) (1622118,)


In [ ]:
def clean_data(df):
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna()
    return df


def train_arima_model(train_data):
    model = ARIMA(train_data, order=(5, 1, 0))
    model_fit = model.fit()
    return model_fit


def train_exponential_smoothing_model(train_data):
    model = ExponentialSmoothing(train_data, trend='add', seasonal=None, damped_trend=False)
    model_fit = model.fit()
    return model_fit


def train_simple_exponential_smoothing_model(train_data):
    model = SimpleExpSmoothing(train_data)
    model_fit = model.fit()
    return model_fit


def autoreg_model(y_train, y_test, lags=1):
    model = AutoReg(y_train, lags=lags)
    model_fitted = model.fit()
    return model_fitted


def calculate_metrics(y_test, y_pred):
    if len(y_test) != len(y_pred):
        raise ValueError("y_test and y_pred must have the same length.")
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}


def train_and_evaluate_autoreg_poisson(y_train, y_test, lags=1):
    # Fit AutoReg model
    model = AutoReg(y_train, lags=lags)
    model_fitted = model.fit()

    # Predictions for training and testing
    y_train_pred = model_fitted.predict(start=len(y_train) - len(y_train), end=len(y_train) - 1, dynamic=False)
    y_test_pred = model_fitted.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1, dynamic=False)

    # Create DataFrames for cleaning
    df_train = pd.DataFrame({'target': y_train, 'predicted': y_train_pred})
    df_test = pd.DataFrame({'predicted': y_test_pred})

    # Clean DataFrames
    df_train = clean_data(df_train)
    df_test_clean = clean_data(df_test)

    # Check for missing or infinite values in the cleaned test DataFrame
    if df_test_clean[['predicted']].isnull().values.any() or np.isinf(df_test_clean[['predicted']]).values.any():
        raise ValueError("Data contains missing or infinite values after cleaning")

    # Fit Poisson Model
    poisson_model = Poisson(df_train['target'], df_train[['predicted']])

    try:
        poisson_fitted = poisson_model.fit()
    except Exception as e:
        print(f"Poisson Model fitting failed: {e}")
        return None, None

    # Predict using Poisson model (if needed)
    try:
        y_test_pred_poisson = poisson_fitted.predict(exog=df_test_clean[['predicted']])
    except Exception as e:
        print(f"Prediction with Poisson Model failed: {e}")
        return None, None

    # Calculate metrics
    try:
        metrics_poisson_model = calculate_metrics(y_test, y_test_pred_poisson)
    except ValueError as e:
        print(f"Metric calculation failed: {e}")
        metrics_negative_binomial = None

    return poisson_fitted, metrics_poisson_model


def train_and_evaluate_autoreg_negbinom(y_train, y_test, lags=1):
    # Fit AutoReg model
    model = AutoReg(y_train, lags=lags)
    model_fitted = model.fit()

    # Predictions for training and testing
    y_train_pred = model_fitted.predict(start=len(y_train) - len(y_train), end=len(y_train) - 1, dynamic=False)
    y_test_pred = model_fitted.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1, dynamic=False)

    # Create DataFrames for cleaning
    df_train = pd.DataFrame({'target': y_train, 'predicted': y_train_pred})
    df_test = pd.DataFrame({'predicted': y_test_pred})

    # Clean DataFrames
    df_train = clean_data(df_train)
    df_test_clean = clean_data(df_test)

    # Check for missing or infinite values in the cleaned test DataFrame
    if df_test_clean[['predicted']].isnull().values.any() or np.isinf(df_test_clean[['predicted']]).values.any():
        raise ValueError("Data contains missing or infinite values after cleaning")

    # Fit Negative Binomial model
    negbinom_model = NegativeBinomial(df_train['target'], df_train[['predicted']])

    try:
        negbinom_fitted = negbinom_model.fit()
    except Exception as e:
        print(f"Negative Binomial Model fitting failed: {e}")
        return None, None

    # Predict using Negative Binomial model (if needed)
    try:
        y_test_pred_negbinom = negbinom_fitted.predict(exog=df_test_clean[['predicted']])
    except Exception as e:
        print(f"Prediction with Negative Binomial Model failed: {e}")
        return None, None

    # Calculate metrics
    try:
        metrics_negative_binomial = calculate_metrics(y_test, y_test_pred_negbinom)
    except ValueError as e:
        print(f"Metric calculation failed: {e}")
        metrics_negative_binomial = None

    return negbinom_fitted, metrics_negative_binomial

### Model Training

In [ ]:
results = []

# ARIMA Model
arima_model = train_arima_model(y_train)
y_pred_arima = arima_model.forecast(steps=len(y_test))
metrics_arima = calculate_metrics(y_test, y_pred_arima)
metrics_arima['Model Name'] = 'ARIMA'
results.append(metrics_arima)
print('Model: ARIMA', metrics_arima)

# Exponential Smoothing Model
exp_smoothing_model = train_exponential_smoothing_model(y_train)
y_pred_exp_smoothing = exp_smoothing_model.forecast(len(y_test))
metrics_exp_smoothing = calculate_metrics(y_test, y_pred_exp_smoothing)
metrics_exp_smoothing['Model Name'] = 'Exponential Smoothing'
results.append(metrics_exp_smoothing)
print('Model: Exponential Smoothing', metrics_exp_smoothing)

# Simple Exponential Smoothing Model
sim_exp_smoothing_model = train_simple_exponential_smoothing_model(y_train)
y_pred_sim_exp_smoothing = sim_exp_smoothing_model.forecast(len(y_test))
metrics_sim_exp_smoothing = calculate_metrics(y_test, y_pred_sim_exp_smoothing)
metrics_sim_exp_smoothing['Model Name'] = 'Simple Exponential Smoothing'
results.append(metrics_sim_exp_smoothing)
print('Model: Simple Exponential Smoothing', metrics_sim_exp_smoothing)

# Auto Regressive Model
Auto_regressive = autoreg_model(y_train, y_test)
y_pred_autoregressive = Auto_regressive.forecast(len(y_test))
metrics_autoregressive = calculate_metrics(y_test, y_pred_autoregressive)
metrics_autoregressive['Model Name'] = 'AutoRegressive'
results.append(metrics_autoregressive)
print('Model: Auto Regressive', metrics_autoregressive)

# Autoregressive Poisson
poisson_model, metrics_poisson = train_and_evaluate_autoreg_poisson(y_train, y_test)

if poisson_model is not None:
    print('Poisson Model fitted successfully')
    metrics_poisson['Model Name'] = 'Poisson AutoRegressive'
    results.append(metrics_poisson)
    print('Metrics:', metrics_poisson)
else:
    print('Model fitting failed')

# Autoregressive Negative Binomial
negbinom_model, metrics_negative_binomial = train_and_evaluate_autoreg_negbinom(y_train, y_test)

if negbinom_model is not None:
    print('Negative Binomial Model fitted successfully')
    metrics_negative_binomial['Model Name'] = 'Negative Binomial AutoRegressive'
    results.append(metrics_negative_binomial)
    print('Metrics:', metrics_negative_binomial)
else:
    print('Model fitting failed')

Model: ARIMA {'MAE': 57.59324916616699, 'RMSE': 76.1345041904307, 'MAPE': 0.32337730300628326, 'Model Name': 'ARIMA'}
Model: Exponential Smoothing {'MAE': 775943.6526888608, 'RMSE': 895982.1244695284, 'MAPE': 4664.743906121757, 'Model Name': 'Exponential Smoothing'}
Model: Simple Exponential Smoothing {'MAE': 57.750762605991056, 'RMSE': 76.4740212137481, 'MAPE': 0.3222562663255098, 'Model Name': 'Simple Exponential Smoothing'}
Model: Auto Regressive {'MAE': 57.6965729559439, 'RMSE': 72.21592123094705, 'MAPE': 0.37510399287795076, 'Model Name': 'AutoRegressive'}
         Current function value: 2295537725123615551782097815848548389206624168584138027951154452434379335138456208904760376623957187681895358332716401846927826139920108020960022119221035008.000000
         Iterations: 35
Poisson Model fitted successfully
Metrics: {'MAE': 2.3170624008460573e+74, 'RMSE': 2.318645448261235e+74, 'MAPE': 1.4546378017953673e+72, 'Model Name': 'Poisson AutoRegressive'}
         Current function value:

### Save Results

In [ ]:
# convert the results into upto 4 decimals
def format_results(results):
    formatted_results = []
    for result in results:
        formatted_result = {}
        for key, value in result.items():
            if isinstance(value, float):  # Check if the value is a float
                formatted_result[key] = round(value, 4)
            else:
                formatted_result[key] = value  # Keep non-float values as is
        formatted_results.append(formatted_result)
    return formatted_results

# Format the results
formatted_results = format_results(results)

# convert into dataframe
ResultDF = pd.DataFrame(formatted_results)

In [ ]:
ResultDF

,MAE,RMSE,MAPE,Model Name
0,5.759320e+01,7.613450e+01,3.234000e-01,ARIMA
1,7.759437e+05,8.959821e+05,4.664744e+03,Exponential Smoothing
2,5.775080e+01,7.647400e+01,3.223000e-01,Simple Exponential Smoothing
3,5.769660e+01,7.221590e+01,3.751000e-01,AutoRegressive
4,2.317062e+74,2.318645e+74,1.454638e+72,Poisson AutoRegressive
5,2.317062e+74,2.318645e+74,1.454638e+72,Negative Binomial AutoRegressive


In [ ]:
ResultDF.to_csv('/content/drive/MyDrive/Upwork clients/ACCOUNT-CREATOR/DiaTrend Data/Results-NB2-54-50K.csv', index=False)